# 選択した画像データとランダムデータの集合論による類似性の評価

選択してきたかわいいと判断した画像データとランダムに抽出したデータの差異を集合論にうよって評価してみる

1. Simpson係数
2. Dice係数
3. Jaccard係数
のそれぞれを計算して比較

引用元：https://mieruca-ai.com/ai/jaccard_dice_simpson/

### 実際の特徴量データを集合としてデータ整形を行う

In [2]:
# pandasモジュールの読み込み
import pandas as pd
# グラフ描画ライブラリmatplotlibの読み込み
import matplotlib.pyplot as plt
# 数値計算用ライブラリnumpyの読み込み
import numpy as np
# 村上3次元の特徴量データを読み込み
# 引数にindex_col=0を入れなとUnnamed:0が追加されてしまう
# index_colで指定した列をスタートで考えてくれる
df = pd.read_csv('../CSV_DATA/arange_feature_category.csv',index_col=0)

In [3]:
# データ全体の概観
df.head()

,name,Face_shape,Hair_len,Eyelid,Eye_shape,Nose_shape,Nose_len,Nose_bridge,Eyebrows,Lips,Mouse,Tears_bag,Dis_eye_brows,Forehead,Barb,Gills,Dis_nose_mouse,Value_of_murakami,Value_of_yoshi,Cloth_color
0,sample1,卵,ロング,二重,アーモンド形,NaN,NaN,高い,細い,ふつう２,ふつう３,ある,近い,広い,ふつう６,ある１,近い１,1,0.0,柄
1,sample2,卵,ロング,二重,アーモンド形,NaN,NaN,高い,太い,ふつう２,小さい,ある,近い,広い,とがっている,ある１,近い１,1,0.0,柄
2,sample3,面長,ロング,二重,アーモンド形,NaN,NaN,高い,ふつう１,厚い,大きい,ない,近い,ふつう５,とがっている,ない２,近い１,1,1.0,白
3,アドリアナ・リマ,卵,ロング,二重,アーモンド形,NaN,NaN,高い,細い,厚い,大きい,ある,近い,広い,丸い,ある１,近い１,1,0.0,黒
4,アンジェラ・ベイビー,逆三角,ロング,二重,アーモンド形,NaN,NaN,高い,ふつう１,厚い,ふつう３,ない,近い,広い,丸い,ある１,近い１,1,1.0,黒


In [4]:
# 村上が評価した画像データ
murakami_value_data = df[:49]
# 吉井が評価した画像データ
yoshi_value_data = df[50:98]
# ランダムに評価された特徴量データ
rand_value_data = df[100:206]

In [5]:
# 村上評価のデータの概観
murakami_value_data.head()

,name,Face_shape,Hair_len,Eyelid,Eye_shape,Nose_shape,Nose_len,Nose_bridge,Eyebrows,Lips,Mouse,Tears_bag,Dis_eye_brows,Forehead,Barb,Gills,Dis_nose_mouse,Value_of_murakami,Value_of_yoshi,Cloth_color
0,sample1,卵,ロング,二重,アーモンド形,NaN,NaN,高い,細い,ふつう２,ふつう３,ある,近い,広い,ふつう６,ある１,近い１,1,0.0,柄
1,sample2,卵,ロング,二重,アーモンド形,NaN,NaN,高い,太い,ふつう２,小さい,ある,近い,広い,とがっている,ある１,近い１,1,0.0,柄
2,sample3,面長,ロング,二重,アーモンド形,NaN,NaN,高い,ふつう１,厚い,大きい,ない,近い,ふつう５,とがっている,ない２,近い１,1,1.0,白
3,アドリアナ・リマ,卵,ロング,二重,アーモンド形,NaN,NaN,高い,細い,厚い,大きい,ある,近い,広い,丸い,ある１,近い１,1,0.0,黒
4,アンジェラ・ベイビー,逆三角,ロング,二重,アーモンド形,NaN,NaN,高い,ふつう１,厚い,ふつう３,ない,近い,広い,丸い,ある１,近い１,1,1.0,黒


In [6]:
# ランダム抽出のデータの概観
rand_value_data.head()

,name,Face_shape,Hair_len,Eyelid,Eye_shape,Nose_shape,Nose_len,Nose_bridge,Eyebrows,Lips,Mouse,Tears_bag,Dis_eye_brows,Forehead,Barb,Gills,Dis_nose_mouse,Value_of_murakami,Value_of_yoshi,Cloth_color
100,1,丸,ロング,二重,アーモンド形,NaN,NaN,普通,ふつう１,ふつう２,小さい,ない,NaN,ふつう５,丸い,ある１,近い１,0,0.0,水色
101,2,丸,ロング,NaN,NaN,NaN,ローマ鼻,低い,ふつう１,NaN,大きい,ない,近い,広い,丸い,ある１,近い１,1,0.0,白
102,3,卵,ミディアム,奥二重,アーモンド形,NaN,NaN,普通,細い,ふつう２,大きい,ない,ふつう４,広い,ふつう６,ない２,近い１,0,0.0,茶色
103,4,NaN,ロング,NaN,NaN,団子鼻,NaN,普通,NaN,NaN,ふつう３,ない,ふつう４,わからない,丸い,ある１,近い１,0,1.0,紺
104,5,面長,ロング,NaN,アーモンド形,団子鼻,NaN,低い,細い,ふつう２,小さい,ない,近い,広い,ふつう６,ある１,近い１,1,0.0,黒


In [7]:
# 評価に関係ない列の削除
murakami_value_data = murakami_value_data.drop(['name','Value_of_yoshi','Value_of_murakami','Cloth_color'],axis=1)

# 村上がかわいいと判断した部分だけ抽出
rand_value_data = rand_value_data[rand_value_data['Value_of_murakami']=='1']
# 上の処理をもとに評価に関係ない列を削除
rand_value_data = rand_value_data.drop(['name','Value_of_yoshi','Cloth_color','Value_of_murakami'],axis=1)

# 村上評価データの欠損値を-1に置換
murakami_value_data = murakami_value_data.fillna(-1)
# ランダム抽出データの欠損値を-1に置換
rand_value_data = rand_value_data.fillna(-1)

In [8]:
murakami_value_data.head()

,Face_shape,Hair_len,Eyelid,Eye_shape,Nose_shape,Nose_len,Nose_bridge,Eyebrows,Lips,Mouse,Tears_bag,Dis_eye_brows,Forehead,Barb,Gills,Dis_nose_mouse
0,卵,ロング,二重,アーモンド形,-1,-1,高い,細い,ふつう２,ふつう３,ある,近い,広い,ふつう６,ある１,近い１
1,卵,ロング,二重,アーモンド形,-1,-1,高い,太い,ふつう２,小さい,ある,近い,広い,とがっている,ある１,近い１
2,面長,ロング,二重,アーモンド形,-1,-1,高い,ふつう１,厚い,大きい,ない,近い,ふつう５,とがっている,ない２,近い１
3,卵,ロング,二重,アーモンド形,-1,-1,高い,細い,厚い,大きい,ある,近い,広い,丸い,ある１,近い１
4,逆三角,ロング,二重,アーモンド形,-1,-1,高い,ふつう１,厚い,ふつう３,ない,近い,広い,丸い,ある１,近い１


In [9]:
rand_value_data.head()

,Face_shape,Hair_len,Eyelid,Eye_shape,Nose_shape,Nose_len,Nose_bridge,Eyebrows,Lips,Mouse,Tears_bag,Dis_eye_brows,Forehead,Barb,Gills,Dis_nose_mouse
101,丸,ロング,-1,-1,-1,ローマ鼻,低い,ふつう１,-1,大きい,ない,近い,広い,丸い,ある１,近い１
104,面長,ロング,-1,アーモンド形,団子鼻,-1,低い,細い,ふつう２,小さい,ない,近い,広い,ふつう６,ある１,近い１
105,-1,ロング,-1,-1,団子鼻,-1,低い,-1,-1,-1,ない,-1,わからない,丸い,-1,-1
108,面長,ショート,二重,細い目,-1,-1,普通,細い,ふつう２,ふつう３,ない,ふつう４,広い,ふつう６,ある１,近い１
109,卵,ショート,二重,猫目,-1,-1,普通,細い,ふつう２,小さい,ない,ふつう４,広い,丸い,ある１,近い１


In [10]:
# データを重複を許して格納する集合を作成
murakami_value_set = []
# -1（欠損値）は集合の要素にいれない
for index, row in murakami_value_data.iterrows():
    for key,data in row.items():
        if data != -1:
            murakami_value_set.append(data)

# 上と同じ操作
rand_value_set = []
for index, row in rand_value_data.iterrows():
    for key,data in row.items():
        if data != -1:
            rand_value_set.append(data)

In [11]:
# print(murakami_value_set)
# print(rand_value_set)

## 類似度計算の実施（Jaccard係数、Dice係数、Simpson係数）

In [12]:
#Jaccard係数(overlap coefficiant)の計算アルゴリズム
def similarity_coefficient(list_a,list_b):
    #集合Aと集合Bの積集合(set型)を作成
    set_intersection = set.intersection(set(list_a), set(list_b))
    #集合Aと集合Bの積集合の要素数を取得
    num_intersection = len(set_intersection)

    #集合Aと集合Bの和集合(set型)を作成
    set_union = set.union(set(list_a), set(list_b))
    #集合Aと集合Bの和集合の要素数を取得
    num_union = len(set_union)
    
    #集合Aの要素数を取得
    num_listA = len(list_a)
    #集合Bの要素数を取得
    num_listB = len(list_b)

    #積集合の要素数を和集合の要素数で割って
    #Jaccard係数を算出
    try:
        return float(num_intersection) / num_union,float(2.0 * num_intersection) / (num_listA + num_listB),float(num_intersection) / min([num_listA,num_listB])
    except ZeroDivisionError:
        return 1.0	

if __name__== '__main__':
    list_a = murakami_value_set
    list_b = rand_value_set

    jaccard,dice,simpson = similarity_coefficient(list_a,list_b)	#各係数を計算
   
    #計算結果を出力　
    print('Jaccard係数による類似度：{0}%'.format(jaccard*100))
    print('Dice係数による類似度：{0}%'.format(dice*100))
    print('Simpson係数による類似度：{0}%'.format(simpson*100))

Jaccard係数による類似度：89.58333333333334%
Dice係数による類似度：7.184628237259815%
Simpson係数による類似度：8.582834331337326%
